## FEVER: Fact Extraction and VERification

In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd
import os
import json

from tqdm import tqdm
from collections import Counter
from itertools import product
from sklearn.linear_model import LogisticRegression

import fever
import utils

/raid/nlp/sameer/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/raid/nlp/sameer/anaconda3/envs/myenv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 5416537/5416537 [02:33<00:00, 35385.49it/s]


### Functionalities of Oracle Class

In [2]:
oracle = fever.Oracle()

In [3]:
from datasets import load_dataset

fever_wiki = load_dataset("fever", 'wiki_pages')

wiki_id2lines = {}
for dump in tqdm(fever_wiki['wikipedia_pages']):
    if len(dump['text']) != 0:
        wiki_id2lines[dump['id']] = [dump['text'], dump['lines']]

100%|██████████| 5416537/5416537 [02:42<00:00, 33424.89it/s]


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('/raid/nlp/sameer/fever/sbert_model/distilroberta')

# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
   model = model.to(torch.device("cuda:1"))

# End to End retrieval

In [4]:
from datasets import load_dataset
fever_wiki = load_dataset("fever", 'wiki_pages')
# fever_nli = load_dataset("fever", 'v1.0')

In [12]:
import json
with open('/raid/nlp/sameer/fever/data/fever-data/train.jsonl') as f:
    fever_nli = [json.loads(line) for line in f]

with open('/raid/nlp/sameer/fever/data/fever-data/shared_task_test.jsonl') as f:
    test_data = [json.loads(line) for line in f]

with open('/raid/nlp/sameer/fever/data/fever-data/dev.jsonl') as f:
    dev_data = [json.loads(line) for line in f]

tfidf_level_1 = open('/raid/nlp/sameer/fever/tfidf_pred_level_1_test.json')
level_1 = json.load(tfidf_level_1)


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('/raid/nlp/sameer/fever/sbert_model/distilroberta')

# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
   model = model.to(torch.device("cuda:1"))

In [14]:
query2pred = {}
invalid_id = []
for sample in tqdm(dev_data):
    try:
        query = sample['claim']
        # pred_sent = oracle.choose_sents_from_doc_ids(query, oracle.closest_docs(query, k=5), k=50)
        pred_sent = oracle.choose_sents_from_doc_ids(query, level_1[query][:5], k=100)

        query2pred[sample['id']] = [[i[0],i[1]] for i in list(pred_sent.keys())]
    except:
        invalid_id.append(sample)

 45%|████▍     | 4457/9999 [09:20<11:04,  8.34it/s]

 45%|████▍     | 4470/9999 [09:22<10:21,  8.90it/s]

In [9]:
len(invalid_id)

471

In [6]:
def return_line_corpus(pred_docs_ids):
    line_corpus = []
    wiki2lines = {}
    for doc_id in pred_docs_ids:
        try:
            lines = wiki_id2lines[doc_id][1].split("\n")
        except:
            continue
        for i in range(len(lines)):
            text = lines[i].replace("\t"," ").strip(",. ")
            text = re.sub(r'\s+', ' ', text)
            text = " ".join(text.split()[1:])
            wiki2lines[text] = [doc_id, i]
            if text.isnumeric()==True or len(text)==0:
                continue
            line_corpus.append(text)
    return(line_corpus, wiki2lines)

In [13]:
query2pred = {}

from rank_bm25 import BM25Okapi
import re
# query2lines = {}

for sample in tqdm(test_data):
    line_corpus, wiki2lines = return_line_corpus(level_1[sample['claim']])
    tokenized_corpus = [doc.split(" ") for doc in line_corpus]

    bm25 = BM25Okapi(tokenized_corpus)
    query = sample['claim']
    tokenized_query = query.split(" ")

    retrieved_lines = bm25.get_top_n(tokenized_query, line_corpus, n=50)
    for line in retrieved_lines:
        if sample['id'] not in query2pred:
            query2pred[sample['id']] = [wiki2lines[line]]
        else:
            query2pred[sample['id']].append(wiki2lines[line])

100%|██████████| 19998/19998 [59:04<00:00,  5.64it/s]  


In [15]:
print(len(query2pred))

19998


In [14]:
with open("test_tfidfboth_level_5_50.json", "w") as outfile: 
    json.dump(query2pred, outfile)

In [19]:
list(query2pred.items())[0]

(89296,
 [['Dimitris', 0],
  ['Yannis', 3],
  ['List_of_alumni_of_Trinity_College,_Cambridge', 2],
  ['List_of_Greek_exonyms_in_Turkey', 29],
  ['List_of_Greek_exonyms_in_Turkey', 28],
  ['List_of_Greek_exonyms_in_Turkey', 27],
  ['List_of_Greek_exonyms_in_Turkey', 30],
  ['List_of_Greek_exonyms_in_Turkey', 15],
  ['List_of_Greek_exonyms_in_Turkey', 33],
  ['List_of_Greek_exonyms_in_Turkey', 32],
  ['List_of_Greek_exonyms_in_Turkey', 23],
  ['List_of_Greek_exonyms_in_Turkey', 26],
  ['List_of_Greek_exonyms_in_Turkey', 16],
  ['List_of_Greek_exonyms_in_Turkey', 13],
  ['List_of_Greek_exonyms_in_Turkey', 12],
  ['List_of_Greek_exonyms_in_Turkey', 17],
  ['List_of_Greek_exonyms_in_Turkey', 21],
  ['List_of_Greek_exonyms_in_Turkey', 31],
  ['List_of_Greek_exonyms_in_Turkey', 25],
  ['List_of_Greek_exonyms_in_Turkey', 9],
  ['List_of_Greek_exonyms_in_Turkey', 10],
  ['List_of_Greek_exonyms_in_Turkey', 20],
  ['List_of_Greek_exonyms_in_Turkey', 18],
  ['List_of_alumni_of_Trinity_College,_Cam

In [5]:
import json
with open('/raid/nlp/sameer/fever/data/fever-data/dev.jsonl') as f:
    data = [json.loads(line) for line in f]

In [48]:
data[4]['evidence'][2][0], len(data), data[0]

([131371, 146150, 'Telemundo', 4],
 9999,
 {'id': 91198,
  'verifiable': 'NOT VERIFIABLE',
  'label': 'NOT ENOUGH INFO',
  'claim': 'Colin Kaepernick became a starting quarterback during the 49ers 63rd season in the National Football League.',
  'evidence': [[[108548, None, None, None]]]})

In [6]:
import re

gold_docs = []
id2gold_docs = {}
id2gold_lines = {}
for i in range(0,len(fever_nli)):
    if fever_nli[i]['label'] == 'NOT ENOUGH INFO':
        continue
    gold = []
    gold_line = []
    for j in range(len(fever_nli[i]['evidence'])):
        for k in range(len(fever_nli[i]['evidence'][j])):
            if fever_nli[i]['evidence'][j][k][2]:
                gold.append(fever_nli[i]['evidence'][j][k][2])
            if fever_nli[i]['evidence'][j][k][3]:
                try:
                    text = wiki_id2lines[fever_nli[i]['evidence'][j][k][2]][1].split("\n")[fever_nli[i]['evidence'][j][k][3]]
                    text = text.replace("\t", " ")
                    text = re.sub(r'\s+', ' ', text)
                    gold_line.append(text)
                except:
                    continue
    if len(gold):
        id2gold_docs[fever_nli[i]['claim']] = set(gold)
    if len(gold_line):
        id2gold_lines[fever_nli[i]['claim']] = set(gold_line)
# gold_docs = [list(set(i)) for i in gold_docs]

In [6]:
cnt = 0
valid_total = 0
query2pred_docs = {}
for i in tqdm(range(0,len(test_data))):
    # if data[i]['label'] == 'NOT ENOUGH INFO':
        query = test_data[i]['claim']
    
        # if query in id2gold_docs:
            # valid_total += 1
        pred_docs = oracle.closest_docs(query, k=100)
        query2pred_docs[query] = pred_docs
        # if len(set(pred_docs).intersection(id2gold_docs[query])):
        #     cnt += 1
# print(cnt/valid_total)

100%|██████████| 19998/19998 [55:37<00:00,  5.99it/s] 


In [7]:
with open("tfidf_pred_level_1_test.json", "w") as outfile: 
    json.dump(query2pred_docs, outfile)

In [1]:
import re

def retrieve(model, line_corpus, query, top_k):
    pred_lines = []
    corpus_embeddings = model.encode(line_corpus, convert_to_tensor=True).to("cuda:1")
    query_embedding = model.encode(query, convert_to_tensor=True).to("cuda:1")
    top_k = min(3, len(line_corpus))

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    for score, idx in zip(top_results[0], top_results[1]):
        pred_lines.append(line_corpus[idx])
    return(pred_lines)

def return_line_corpus(pred_docs_ids):
    line_corpus = []
    wiki2lines = {}
    for doc_id in pred_docs_ids:
        try:
            lines = wiki_id2lines[doc_id][1].split("\n")
        except:
            continue
        for i in range(len(lines)):
            text = lines[i].replace("\t"," ").strip(",. ")
            text = re.sub(r'\s+', ' ', text)
            text = " ".join(text.split()[1:])
            wiki2lines[text] = [doc_id, i]
            if text.isnumeric()==True or len(text)==0:
                continue
            line_corpus.append(text)
    return(line_corpus, wiki2lines)


In [22]:
import re

cnt = 0
valid_total = 0
query2pred_lines = {}
for i in tqdm(range(0,len(fever_nli))):
    query = data[i]['claim']
    if query in id2gold_docs:
        valid_total += 1
        pred_docs_ids = oracle.closest_docs(query, k=3)
        line_corpus = return_line_corpus(pred_docs_ids)
        if len(line_corpus) == 0:
            continue
        pred_lines = retrieve(model, line_corpus, query, 3)
        query2pred_lines[query] = pred_lines
        # for j in range(len(data[i]['evidence'])):
        #     for k in range(len(data[i]['evidence'][j])):
        #         if data[i]['evidence'][j][k][2]:
        #             gold.append(data[i]['evidence'][j][k][3])
        # if len(set(pred_docs).intersection(id2gold_docs[query])):
        #     cnt += 1
# print(cnt/valid_total)

  0%|          | 0/145449 [00:00<?, ?it/s]


NameError: name 'id2gold_docs' is not defined

# SBert Zero shot first stage

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
   model = model.to(torch.device("cuda:1"))
print(model.device)

In [ ]:
import numpy as np
import faiss

with open('/raid/nlp/sameer/fever_old/embeddings.npy', 'rb') as f:
    embeddings = np.load(f)

In [ ]:
corpus = fever_wiki['wikipedia_pages']["text"][1:]
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(x=embeddings[1:], ids=np.array([i for i in range(len(corpus))]))

In [ ]:
D, I = index.search(np.array([query_embeddings[0]]), k=10)

In [ ]:
cnt = 0
valid_total = 0
for i in tqdm(range(0,len(data))):
    if data[i]['claim'] in id2gold_docs:
        valid_total += 1
        pred_docs = oracle.closest_docs(data[i]['claim'], k=10)
        if len(set(pred_docs).intersection(id2gold_docs[data[i]['claim']])):
            cnt += 1
print(cnt/valid_total)

### Dataset Structure